Install and import packages, mount **drive**

In [1]:
!pip install --user simpletransformers

     |████████████████████████████████| 249 kB 4.1 MB/s 
     |████████████████████████████████| 1.8 MB 53.2 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 342 kB 48.8 MB/s 
     |████████████████████████████████| 4.2 MB 38.0 MB/s 
     |████████████████████████████████| 6.6 MB 29.7 MB/s 
     |████████████████████████████████| 10.1 MB 40.8 MB/s 
     |████████████████████████████████| 1.2 MB 42.9 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
     |████████████████████████████████| 145 kB 50.3 MB/s 
     |████████████████████████████████| 181 kB 52.9 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 136 kB 51.4 MB/s 
     |████████████████████████████████| 212 kB 39.9 MB/s 
     |████████████████████████████████| 1.1 MB 47.2 MB/s 
     |████████████████████████████████| 127 kB 50.2 MB/s 
     |██████████████

In [1]:
!pip install --user transformers

In [3]:
!sudo pip install checklist --user

  Using cached checklist-0.0.11.tar.gz (12.1 MB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached patternfork_nosql-3.6.tar.gz (22.3 MB)
  Using cached iso-639-0.4.5.tar.gz (167 kB)
     |████████████████████████████████| 81 kB 3.9 MB/s 
     |████████████████████████████████| 5.6 MB 16.3 MB/s 
     |████████████████████████████████| 5.6 MB 36.5 MB/s 
     |████████████████████████████████| 419 kB 48.4 MB/s 
     |████████████████████████████████| 104 kB 53.5 MB/s 
     |████████████████████████████████| 3.6 MB 44.0 MB/s 
  Created wheel for checklist: filename=checklist-0.0.11-py3-none-any.whl size=12165635 sha256=a35ec76f955c767a16dd63f134d054cfe2fef2e0a85499d68919580b4589d274
  Stored in directory: /root/.cache/pip/wheels/6a/8a/07/6446879be434879c27671c83443727d74cecf6b630c8a24d03
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=169061 sha256=1d347428448c5c4cd91b86cd06f239b0234f3bf65e0fe9501b808c12fd477b74
  Stored in directory: /root/.

In [4]:
!pip install --user tabulate

In [5]:
!pip install --user spacy

In [7]:
import numpy as np
import pandas as pd
import torch
import os
import logging
import spacy
import checklist
from checklist.perturb import Perturb
from checklist.editor import Editor
editor = Editor()
import random
random.seed(0)

from simpletransformers.classification import ClassificationModel, ClassificationArgs

from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# 0. Preparation and definitions

**Import the data**

In [8]:
# # Import the data
# train = pd.read_csv('../input/olid-data/olid-train.csv')
# test = pd.read_csv('../input/olid-data/olid-test.csv')

# Import the data
train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/olid-train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/olid-test.csv')
subset_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/olid-subset-diagnostic-tests.csv')

**Making the evaluation call; precision, recall and F1**

In [9]:
def evaluation(df, freq_0, freq_1):
    df['TP'] = (df['labels'] == 1) & (df['labels'] == df['predictions'])
    df['FN'] = (df['labels'] == 1) & (df['labels'] != df['predictions'])
    df['FP'] = (df['labels'] == 0) & (df['labels'] != df['predictions'])
    df['TN'] = (df['labels'] == 0) & (df['labels'] == df['predictions'])

    precision_1 = sum(df['TP']) / (sum(df['TP']) + sum(df['FP'])) if (sum(df['TP']) + sum(df['FP']) > 0) else 0
    precision_0 = sum(df['TN']) / (sum(df['FN']) + sum(df['TN'])) if (sum(df['FN']) + sum(df['TN']) > 0) else 0
    precision_avg = np.mean([precision_1, precision_0])
    precision_wavg = freq_0 * precision_0 + freq_1 * precision_1

    recall_1 = sum(df['TP']) / (sum(df['TP']) + sum(df['FN'])) if (sum(df['TP']) + sum(df['FN']) > 0) else 0
    recall_0 = sum(df['TN']) / (sum(df['FP']) + sum(df['TN'])) if (sum(df['TP']) + sum(df['FN']) > 0) else 0
    recall_avg = np.mean([recall_1, recall_0])
    recall_wavg = freq_0 * recall_0 + freq_1 * recall_1

    F1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1) if (precision_1 + recall_1 > 0) else 0
    F1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0) if (precision_0 + recall_0 > 0) else 0
    F1_avg = np.mean([F1_1, F1_0])
    F1_wavg = freq_0 * F1_0 + freq_1 * F1_1

    print('metric, class_1, class_0, avg, wavg')
    print("precision: ", precision_1, precision_0, precision_avg, precision_wavg)
    print("recall: ", recall_1, recall_0, recall_avg, recall_wavg)
    print("F1: ", F1_1, F1_0, F1_avg, F1_wavg)

# 1. Class distributions (1 point)

In [10]:
# 1. Class distributions (1 point)
print(train['labels'].value_counts())
print(train['labels'].value_counts(normalize=True))
freq_0 = train['labels'].value_counts(normalize=True).iloc[0]
freq_1 = train['labels'].value_counts(normalize=True).iloc[1]
print(train[train['labels'] == 0].iloc[0]['text'])
print(train[train['labels'] == 1].iloc[0]['text'])

0    8840
1    4400
Name: labels, dtype: int64
0    0.667674
1    0.332326
Name: labels, dtype: float64
Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT
@USER She should ask a few native Americans what their take on this is.


# 2.	Baselines (1 point) 

In [11]:
# Random
df_random = test[['text', 'labels']]
predictions = []
for i in range(len(df_random)):
    if random.random() > 0.5:
        predictions.append(1)
    else:
        predictions.append(0)
df_random['predictions'] = predictions
evaluation(df_random, freq_0, freq_1)

metric, class_1, class_0, avg, wavg
precision:  0.26823529411764707 0.7103448275862069 0.48929006085192694 0.5634202092129694
recall:  0.475 0.49838709677419357 0.4866935483870968 0.49061494980996007
F1:  0.3428571428571429 0.585781990521327 0.4643195666892349 0.5050516786087582


In [12]:
# Majority
df = train[['text', 'labels']]
majority_class = df['labels'].value_counts().idxmax()
df_majority = test[['text', 'labels']]
predictions = [majority_class] * len(df_majority)
df_majority['predictions'] = predictions
evaluation(df_majority, freq_0, freq_1)

metric, class_1, class_0, avg, wavg
precision:  0 0.7209302325581395 0.36046511627906974 0.4813461673575493
recall:  0.0 1.0 0.5 0.6676737160120846
F1:  0 0.8378378378378378 0.4189189189189189 0.5594023026047195


# 3.	Classification by fine-tuning BERT (2.5 points)

In [13]:
# Preparing train data
train_df = train.iloc[:10000,:][['text','labels']] #+/- 80% of the training set
# Preparing eval data
eval_df = train.iloc[10000:,:][['text','labels']] #+/- 20% of the training set
# Preparing test data
test_df = test[['text','labels']]
test_list = test_df['text'].values.tolist()

print(len(train_df))
# print(train_df.head())
# print(len(eval_df))
# print(eval_df.head())
# print(len(test_list))
# print(test_list[:2])

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = ClassificationArgs()
model_args.overwrite_output_dir = True

cuda_available = torch.cuda.is_available()
print(cuda_available)
model = ClassificationModel(
    "bert", "bert-base-cased", use_cuda=cuda_available, args=model_args
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Make predictions with the model
predictions, raw_outputs = model.predict(test_list)

10000
True


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/405 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.5420739062535377, 'tp': 713, 'tn': 1880, 'fp': 287, 'fn': 360, 'auroc': 0.851951947173372, 'auprc': 0.7603344674572005, 'eval_loss': 0.44266449728129825}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [14]:
# Attach predictions to test df for evaluation
test_df['predictions'] = predictions
print(test_df)

# Calculate evaluation metrics
evaluation(test_df, freq_0, freq_1)

print(sum(test_df['predictions'] == test_df['labels']))
print(len(test_df['predictions']))

                                                  text  labels  predictions
0    #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...       1            1
1    #ConstitutionDay is revered by Conservatives, ...       0            0
2    #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...       0            0
3    #Watching #Boomer getting the news that she is...       0            0
4    #NoPasaran: Unity demo to oppose the far-right...       1            0
..                                                 ...     ...          ...
855  #DespicableDems lie again about rifles. Dem Di...       1            0
856  #MeetTheSpeakers 🙌 @USER will present in our e...       0            0
857  3 people just unfollowed me for talking about ...       1            1
858  #WednesdayWisdom Antifa calls the right fascis...       0            0
859      #Kavanaugh typical #liberals , #Democrats URL       0            0

[860 rows x 3 columns]
metric, class_1, class_0, avg, wavg
precision:  0.73658536585365

In [15]:
# Confusion matrix elements
print('TP: ', sum(test_df['TP']))
print('FN: ', sum(test_df['FN']))
print('FP: ', sum(test_df['FP']))
print('TN: ', sum(test_df['TN']))

TP:  151
FN:  89
FP:  54
TN:  566


# 4.	Inspect the tokenization of the OLIDv1 training set using the BERT’s tokenizer (2.5 points)

In [16]:
train_text = train['text'].values.tolist()
#print(train_text)
train_tokens = []
for i in range(len(train_text)):
  tokens = model.tokenizer.tokenize(train_text[i])
  train_tokens.extend(tokens)

In [17]:
# number of tokens
print(len(train_tokens))

# number of token split into subwords
train_tokens_str = ' '.join(train_tokens)
print(train_tokens_str.count('##'))
train_tokens_str[:10000]

478955
91024


'@ US ##ER She should ask a few native Americans what their take on this is . @ US ##ER @ US ##ER Go home you ’ re drunk ! ! ! @ US ##ER # MA ##GA # Trump ##20 ##20 [UNK] U ##RL Amazon is investigating Chinese employees who are selling internal data to third - party seller ##s looking for an edge in the competitive marketplace . U ##RL # Amazon # MA ##GA # K ##AG # CH ##IN ##A # T ##CO ##T @ US ##ER Someone should \' ve ##T ##ake ##n " this piece of shit to a volcano . [UNK] " @ US ##ER @ US ##ER Obama wanted liberal ##s & am ##p ; illegal ##s to move into red states @ US ##ER Liberals are all Ko ##oko ##o ! ! ! @ US ##ER @ US ##ER Oh no ##es ! To ##ugh shit . @ US ##ER was literally just talking about this lo ##l all mass shooting ##s like that have been set ups . it ’ s propaganda used to divide us on major issues like gun control and terrorism @ US ##ER Buy more ice ##cre ##am ! ! ! @ US ##ER Canada doesn ’ t need another C ##UC ##K ! We already have enough # Lo ##oney ##L ##ef ##t 

In [18]:
# How long (in characters) is the longest subword in the BERT’s vocabulary? (0.5 points)
print(max(list(model.tokenizer.vocab.keys()), key=len))
print(len(max(list(model.tokenizer.vocab.keys()), key=len)))

telecommunications
18


# 5.	Typos (6 points) 

In [19]:
# reseed
np.random.seed(42)

# text
subset_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/olid-subset-diagnostic-tests.csv')
text = subset_test['text'].tolist()

In [20]:
# overload existing method from checklist to add more than 1 typo
def add_typos(string, typos=5):
        """Perturbation functions, swaps random characters with their neighbors
        Parameters
        ----------
        string : str
            input string
        typos : int
            number of typos to add
        Returns
        -------
        list(string)
            perturbed strings
        """
        string = list(string)
        swaps = np.random.choice(len(string) - 1, typos)
        for swap in swaps:
            tmp = string[swap]
            string[swap] = string[swap + 1]
            string[swap + 1] = tmp
        return ''.join(string)

In [21]:
# add_typos
typos = Perturb.perturb(text, add_typos)
#print(typos.data)

# plug back into pandas
new_text = []
for t in typos.data:
  new_text.append(t[1])
subset_test['text_typos'] = new_text
print(subset_test)

       id                                               text  labels  \
0   89200      @USER @USER Who the hell does he think he is?       1   
1   71294  #BREAKING. #Greece: Molotov cocktails fly afte...       1   
2   55633  #OrrinHatch I can’t believe this sexist , clue...       1   
3   16856  @USER @USER I'll use that one the next time im...       1   
4   26072    0-1 lost my acca on the first fucking fight cba       1   
..    ...                                                ...     ...   
95  45518  @USER He is obviously getting suspended. He is...       0   
96  51610  #Canada - EXCLUSIVE: #Trudeau #Liberals leave ...       0   
97  26758  @USER @USER ...than why did you show us how ho...       0   
98  30718  @USER @USER @USER You have yet to answer what ...       0   
99  82558  #MAGA  ... got any ideas how she could have do...       0   

                                           text_typos  
0       @USER @SUER Wh othe hell doe she thinkh e si?  
1   #BREAKING. #Greece:

In [22]:
# using model trained previously
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### Perturbed type test data ----------------------------------------------------

# Preparing test data
subset_test_df = subset_test[['text','labels']]
subset_test_list = subset_test_df['text'].values.tolist()

# Make predictions with the model on non-perturbed set
subset_predictions, subset_raw_outputs = model.predict(subset_test_list)
#print(predictions, raw_outputs)

# Attach predictions to non-perturbed df for evaluation
subset_test_df['predictions'] = subset_predictions


### Perturbed type test data ----------------------------------------------------
# Preparing perturbed typo test data
subset_typos_test_df = subset_test[['text_typos','labels']]
subset_typos_test_list = subset_typos_test_df['text_typos'].values.tolist()

# Make predictions with the model on perturbed set
subset_typos_predictions, subset_typos_raw_outputs = model.predict(subset_typos_test_list)
#print(predictions, raw_outputs)

# Attach predictions to typo perturbed df for evaluation
subset_typos_test_df['predictions'] = subset_typos_predictions


# get class distributions for getting weighted F1 score later --------------------
subset_freq_0 = subset_test_df['labels'].value_counts(normalize=True).iloc[0]
subset_freq_1 = subset_test_df['labels'].value_counts(normalize=True).iloc[1]
# note that class distributions are equal in this subset so weighted and macro
# F1 scores will be the same

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

/root/.local/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

/root/.local/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
# Calculate evaluation metrics for non-perturbed data
print('NON-PERTURBED DATASET:')
evaluation(subset_test_df, subset_freq_0, subset_freq_1)
print('Correctly identified messages in data: ' + str(sum(subset_test_df['predictions'] == subset_test_df['labels'])))
print('Total number of messages: ' + str(len(subset_test_df['predictions'])))
print('\n')

# Calculate evaluation metrics for perturbed data
print('PERTURBED TYPO DATASET:')
evaluation(subset_typos_test_df, subset_freq_0, subset_freq_1)
print('Correctly identified messages in typo data: ' + str(sum(subset_typos_test_df['predictions'] == subset_typos_test_df['labels'])))
print('Total number of messages: ' + str(len(subset_typos_test_df['predictions'])))
print('\n')

NON-PERTURBED DATASET:
metric, class_1, class_0, avg, wavg
precision:  0.9428571428571428 0.7384615384615385 0.8406593406593407 0.8406593406593407
recall:  0.66 0.96 0.81 0.81
F1:  0.7764705882352942 0.8347826086956522 0.8056265984654732 0.8056265984654732
Correctly identified messages in data: 81
Total number of messages: 100


PERTURBED TYPO DATASET:
metric, class_1, class_0, avg, wavg
precision:  0.8787878787878788 0.6865671641791045 0.7826775214834916 0.7826775214834916
recall:  0.58 0.92 0.75 0.75
F1:  0.6987951807228915 0.7863247863247863 0.7425599835238389 0.7425599835238389
Correctly identified messages in typo data: 75
Total number of messages: 100




In [24]:
# save dataframes to html
subset_test_df.to_html('/content/gdrive/MyDrive/Colab Notebooks/subset_test_df.html')
subset_typos_test_df.to_html('/content/gdrive/MyDrive/Colab Notebooks/subset_typos_test_df.html')

# read dataframes from html
#subset_test_df = pd.read_html('../input/olid-subset-results/subset_test_df.html', index_col=0)[0]
#subset_typos_test_df = pd.read_html('../input/olid-subset-results/subset_typos_test_df.html', index_col=0)[0]

In [36]:
# The examples below are mislabeled as not containing 
# hate-speech by the model trained on the perturbed dataset:

print(subset_test_df['text'][16])
print(subset_typos_test_df['text_typos'][16])
print('\n')

print(subset_test_df['text'][21])
print(subset_typos_test_df['text_typos'][21])
print('\n')

print(subset_test_df['text'][28])
print(subset_typos_test_df['text_typos'][28])
print('\n')

@USER oh fuck off 😂😂
U@SER hof uc kfof 😂😂


@USER @USER @USER @USER Then why aren’t there so many shootings in Virginia?  Jacksonville shooter got his guns IN MARYLAND!! Among the poster children for gun control!  What good did that do?!  You’re full of crap.
@USER @USER @USER @USER Then why aern’t there so many shootings in Virginia?  Jacksonville shooter got ihs guns IN MARYLAND!! Among the poster children for gun control!  Waht good did that do? ! You’re full of cra.p


#Feinstein you cant say anything truthful.  You are a loers and liar. URL
#Feinstein you cant say anyhting truthful . oYu are a loers adn lair. URL




# 6.	Negation (4.5 points) 

In [26]:
nlp = spacy.load('en_core_web_sm')
pdata = list(nlp.pipe(text))

# reseed
np.random.seed(42)

# text
subset_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/olid-subset-diagnostic-tests.csv')
text = subset_test['text'].tolist()

In [28]:
# add_negations
negations = Perturb.perturb(pdata, Perturb.add_negation)
#print(negations.data)
#print(len(subset_test))
#print(len(negations.data))

ids = []
text = []
text_negations = []
labels = []

# build negations dataframe from scratch since the method uses spacy
for n in negations.data:
    for index, row in subset_test.iterrows():
        if n[0] == row['text']:
            ids.append(row['id'])
            text.append(n[0])            
            labels.append(row['labels'])
            text_negations.append(n[1])

# initialize data of lists.
neg_data = {'id': ids, 'text': text,  'labels': labels, 'text_negations': text_negations}
subset_test = pd.DataFrame(neg_data)
# print(subset_test)

In [29]:
# using model trained previously
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### Perturbed type test data ----------------------------------------------------

# Preparing test data
subset_test_df = subset_test[['text','labels']]
subset_test_list = subset_test_df['text'].values.tolist()

# Make predictions with the model
subset_predictions, subset_raw_outputs = model.predict(subset_test_list)
#print(predictions, raw_outputs)

# Attach predictions to test df for evaluation
subset_test_df['predictions'] = subset_predictions


### Perturbed type test data ----------------------------------------------------

# Preparing negations test data
subset_negs_test_df = subset_test[['text_negations','labels']]
subset_negs_test_list = subset_negs_test_df['text_negations'].values.tolist()

# Make predictions with the model on negations dataset
subset_negs_predictions, subset_negs_raw_outputs = model.predict(subset_negs_test_list)
#print(predictions, raw_outputs)

# Attach predictions to negations test df for evaluation
subset_negs_test_df['predictions'] = subset_negs_predictions

# get class distributions for getting weighted F1 score later
subset_freq_0 = subset_test_df['labels'].value_counts(normalize=True).iloc[0]
subset_freq_1 = subset_test_df['labels'].value_counts(normalize=True).iloc[1]
# note that class distributions are equal in this subset so weighted and macro
# F1 scores will be the same

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

/root/.local/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

/root/.local/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
# Calculate evaluation metrics for non-perturbed data
print('NON-PERTURBED DATASET:')
evaluation(subset_test_df, subset_freq_0, subset_freq_1)
print('Correctly identified messages in data: ' + str(sum(subset_test_df['predictions'] == subset_test_df['labels'])))
print('Total number of messages: ' + str(len(subset_test_df['predictions'])))
print('\n')

# Calculate evaluation metrics for perturbed data
print('PERTURBED TYPO DATASET:')
evaluation(subset_negs_test_df, subset_freq_0, subset_freq_1)
print('Correctly identified messages in negations data: ' + str(sum(subset_negs_test_df['predictions'] == subset_negs_test_df['labels'])))
print('Total number of messages: ' + str(len(subset_negs_test_df['predictions'])))
print('\n')

NON-PERTURBED DATASET:
metric, class_1, class_0, avg, wavg
precision:  0.9428571428571428 0.7384615384615385 0.8406593406593407 0.8406593406593407
recall:  0.66 0.96 0.81 0.81
F1:  0.7764705882352942 0.8347826086956522 0.8056265984654732 0.8056265984654732
Correctly identified messages in data: 81
Total number of messages: 100


PERTURBED TYPO DATASET:
metric, class_1, class_0, avg, wavg
precision:  0.9444444444444444 0.75 0.8472222222222222 0.8472222222222222
recall:  0.68 0.96 0.8200000000000001 0.8200000000000001
F1:  0.7906976744186047 0.8421052631578947 0.8164014687882497 0.8164014687882497
Correctly identified messages in negations data: 82
Total number of messages: 100




In [31]:
# The examples below are mislabeled as not containing 
# hate-speech by the model trained on the perturbed dataset:

print(subset_test_df['text'][30])
print(subset_negs_test_df['text_negations'][30])
print('\n')

print(subset_test_df['text'][31])
print(subset_negs_test_df['text_negations'][31])
print('\n')

print(subset_test_df['text'][71])
print(subset_negs_test_df['text_negations'][71])
print('\n')

@USER @USER @USER @USER Beats the hell NJ out of me lol. Dude already made the assumption that I'm antifa. He even ignored all the evidence to make that assumption.
@USER @USER @USER @USER doesn't beat the hell NJ out of me lol. Dude already made the assumption that I'm antifa. He even ignored all the evidence to make that assumption.


#ConfirmKavanugh now, stall tactics are DC cronies only recourse. No more delays.  #VoteRed to end this madness and to #MAGA
#ConfirmKavanugh now, stall tactics are not DC cronies only recourse. No more delays.  #VoteRed to end this madness and to #MAGA


#antifa #Resist.. Trump is trying to bring world peace, not obstruct like the democrats..  is this good for you also? or haters gonna hate? URL
#antifa #Resist.. Trump is not trying to bring world peace, not obstruct like the democrats..  is this good for you also? or haters gonna hate? URL




In [32]:
# save dataframes to html
#subset_test_df.to_html('subset_test_df.html')
subset_negs_test_df.to_html('/content/gdrive/MyDrive/Colab Notebooks/subset_negs_test_df.html')

# read dataframes from html
#subset_test_df = pd.read_html('../input/olid-subset-results/subset_test_df.html', index_col=0)[0]
#subset_typos_test_df = pd.read_html('../input/olid-subset-results/subset_typos_test_df.html', index_col=0)[0]

# 7. Creating examples from scratch with checklist  (2.5 points)

In [33]:
from checklist.editor import Editor
editor = Editor()

# getting masked language model suggestions
hate = editor.suggest('I hate {mask}.')[:30]
not_hate = editor.suggest('I don\'t hate {mask}.')[:30]

hate_mask = []
for i in hate:
    hate_mask.append('I hate ' + i)
#print(hate_mask)

not_hate_mask = []
for i in hate:
    not_hate_mask.append('I don\'t hate ' + i)
#print(not_hate_mask)

# using the built-in lexicon, we also explore more specific mask subjects
ret1 = editor.template('I hate {nationality}.')
ret2 = editor.template('I don\'t hate {nationality}.')
ret3 = editor.template('I hate {religion}.')
ret4 = editor.template('I don\'t hate {religion}.')

hate_nationalities = list(np.random.choice(ret1.data, 10))
nationalities = list(np.random.choice(ret2.data, 10))
hate_religions = list(np.random.choice(ret3.data, 10))
religions = list(np.random.choice(ret4.data, 10))

# build dataset
hate_list = hate_mask + not_hate_mask + hate_nationalities + nationalities + hate_religions + religions
hate_df = pd.DataFrame({'text': hate_list})
print(hate_df)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

                         text
0                   I hate it
1                 I hate them
2           I hate capitalism
3                 I hate cats
4             I hate politics
..                        ...
95       I don't hate Taoism.
96    I don't hate Shintoism.
97  I don't hate Agnosticism.
98      I don't hate Sikhism.
99       I don't hate Taoism.

[100 rows x 1 columns]


/root/.local/lib/python3.7/site-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


In [34]:
# Make predictions with the model on hate dataset
hate_predictions, hate_raw_outputs = model.predict(hate_list)

# Attach predictions to test df for evaluation
hate_df['predictions'] = hate_predictions

print(hate_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

                         text  predictions
0                   I hate it            0
1                 I hate them            1
2           I hate capitalism            1
3                 I hate cats            1
4             I hate politics            0
..                        ...          ...
95       I don't hate Taoism.            0
96    I don't hate Shintoism.            0
97  I don't hate Agnosticism.            0
98      I don't hate Sikhism.            0
99       I don't hate Taoism.            0

[100 rows x 2 columns]


In [35]:
# save hate result to html
hate_df.to_html('/content/gdrive/MyDrive/Colab Notebooks/hate_df.html')